In [1]:
import os 
import random
import torch
import numpy as np
import PIL
import cv2
import time
from pynput.mouse import Button, Controller
from pynput.keyboard import Listener, KeyCode
import onnx
import onnxruntime as ort
import matplotlib.pyplot as plt
import pygame
import dxcam
import threading

pygame 2.1.2 (SDL 2.0.18, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
TOGGLE_KEY = KeyCode(char="t")
clicking = False

mouse = Controller()

def clicker():
    while True:
        if clicking:
            mouse.click(Button.left,1)
        time.sleep(0.001)
        
def toggle_event(key):
    if key == TOGGLE_KEY:
        global clicking
        clicking = not clicking
        
click_thread = threading.Thread(target=clicker,daemon=True)
click_thread.start()


with Listener(on_press = toggle_event) as listener:
    #listener.join()
    time.sleep()


# listener = Listener(on_press=toggle_event)
# listener.start()

while True:
    pass

KeyboardInterrupt: 

In [2]:
def getFrame(width=640,height=640,verbose=False):
    
    total_time = time.time()
    frame_array = camera.get_latest_frame()
    
    frame = PIL.Image.fromarray(frame_array)

    resized = frame.resize((width,height))
    resized = np.array(resized).astype(np.float32) # Converting to the expected float 32 input
    resized = np.expand_dims(resized.transpose(2,0,1),0) # Setting dimensions to (1,3,640,640)
    resized /= 255 # Normalizing values 
    
    if verbose:
        print(time.time() - total_time)
    
    return frame_array,resized

In [3]:
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model("model.onnx")
ort_sess = ort.InferenceSession('model.onnx', providers=['CUDAExecutionProvider'])

In [ ]:
# Initializing variables

camera = dxcam.create(device_idx=0, output_idx=0)

original_res = np.array(camera.grab()).shape

image_width = 640
image_height = 640

window_width = 1280
window_height = 720

classes_dict = {0: 'GoldenCartridge',
           1: 'GoldenCookie',
           2: 'RedCartridge',
           3: 'RedCookie'}

classes_color_dict = {0: (100,255,0), #Green
           1: (100,255,0), #Green
           2: (255,0,220), #Pink
           3: (255,0,220)} #Pink

font = cv2.FONT_HERSHEY_SIMPLEX

camera.start(target_fps=40)

has_detected = False

# Creating clicker thread 
TOGGLE_KEY = KeyCode(char="t") # Select the toogle key that activates autoclicking
clicking = False

mouse = Controller()

def clicker():
    while True:
        if clicking:
            mouse.click(Button.left,1)
        time.sleep(0.001)
        
def toggle_event(key):
    if key == TOGGLE_KEY:
        global clicking
        clicking = not clicking
        
click_thread = threading.Thread(target=clicker, daemon=True) # A Daemon thread kills it when the main thread ends 
click_thread.start()

# Start the loop

with Listener(on_press = toggle_event) as listener: # Starting the listener thread
    while True:

        total_iter_time = time.time()

        pygame_init_time = time.time()
        pygame.init()
        surface = pygame.display.set_mode((window_width,window_height))
        pygame.display.set_caption("Image")
        pygame_init_time = time.time() - pygame_init_time

        get_screenshoot_time = time.time()
        original,resized = getFrame(image_width, image_height)
        get_screenshoot_time = time.time() - get_screenshoot_time 


        inference_time = time.time()
        outputs = ort_sess.run(None, {'images': resized})
        inference_time = time.time() - inference_time


        img = cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, (window_width,window_height))


        print_time = time.time()
        if len(outputs[0]) > 0:
            x1_all = []
            y1_all = []
            x2_all = []
            y2_all = []
            classes_all = []
            confidences = []

            for i in range(len(outputs[0])):
                x1_all.append(outputs[0][i][1] * window_width / image_width)
                y1_all.append(outputs[0][i][2] * window_height / image_height)
                x2_all.append(outputs[0][i][3] * window_width / image_width)
                y2_all.append(outputs[0][i][4] * window_height / image_height)
                classes_all.append(int(outputs[0][i][5]))
                confidences.append(round(outputs[0][i][6],2))



            for x1,y1,x2,y2,classes,confidence in zip(x1_all,y1_all,x2_all,y2_all,classes_all,confidences):
                cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), classes_color_dict[classes])
                cv2.putText(img,classes_dict[classes] + ": " + str(confidence),(int(x1),int(y1)-8), 
                            font, 0.6,classes_color_dict[classes],2,cv2.LINE_AA)


            x1 = original_res[1] * x1_all[0] / window_width
            x2 = original_res[1] * x2_all[0] / window_width
            y1 = original_res[0] * y1_all[0] / window_height
            y2 = original_res[0] * y2_all[0] / window_height

            #mouse.position = ((x1+x2)/2, (y1+y2)/2)
            has_detected = True

        elif has_detected:
            #mouse.position = (385, 570) # Back to the main cookie
            has_detected = False


        displayImage = pygame.image.frombuffer(img.tobytes(), img.shape[1::-1],"BGR")
        surface.blit(displayImage,(0,0))
        pygame.display.update()
        print_time = time.time() - print_time

        total_iter_time = time.time() - total_iter_time

In [9]:
#mouse.position = (1280, 125) # Open farms
mouse.position = (2075, 510) # Close garden
time.sleep(0.01)
mouse.position = (2075, 285) # Re-open garden
time.sleep(0.01)

x,y = 1885,190 # Top right garden hole coords
for i in range(0,6):
    for j in range(0,6):
        mouse.position = (1005, 295) # Select golden trebol
        time.sleep(0.01)
        mouse.position = (x-i*40, y+j*40) # Travel through the garden holes
        time.sleep(0.01)
        
mouse.position = (1745, 425) # Change fertilizer